### 纪念一下下我的大作业哈

In [10]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import os
import plotly

In [11]:
def var_to_dataframe(var_name,dataframe_columns_list):
    # 变量数据的处理
    data_dict = {i:var_name[i].x for i in var_name}
    df = pd.Series(data_dict).reset_index()
    # df = pd.Series(data_dict).unstack()
    df.columns = [dataframe_columns_list]
    return df

In [43]:
def sensitive_test(i):
    file_path = './final data/所有数据.xlsx'
    # 数据的读取
    data_Indices = pd.read_excel(file_path,sheet_name='Indices')
    data_Cost_matrix = pd.read_excel(file_path,sheet_name='Cost',index_col=0)
    data_Handling_Cost = pd.read_excel(file_path,sheet_name='Handling Cost',index_col=0)
    data_Minimum_Cost_matrix = pd.read_excel(file_path,sheet_name='Minimum Cost',index_col=0)
    data_Time_matrix = pd.read_excel(file_path,sheet_name='Time',index_col=0)
    data_Distance_matrix = pd.read_excel(file_path,sheet_name='Distance',index_col=0)
    data_Customer_Information = pd.read_excel(file_path,sheet_name='Customer Information',index_col=0)
    data_contract_Qualify_Cost = pd.read_excel(file_path,sheet_name='Qualify Cost',index_col=0)  # 合约费率
    data_public_Qualify_Cost = pd.read_excel(file_path,sheet_name='Public Qualify Cost',index_col=0)  # 非合约费率

    # 提取其中的Indices
    raw_Factories = [i for i in data_Indices['Factories'].values.tolist() if str(i)!='nan']
    raw_Depots = [i for i in data_Indices['Depots'].values.tolist() if str(i)!='nan']
    raw_Customers = [i for i in data_Indices['Customers'].values.tolist() if str(i)!='nan']
    raw_Cities = raw_Factories + raw_Depots + raw_Customers

    # 构建matrix类型数据的multidict
    raw_Cost = {(source,destination):data_Cost_matrix.at[source,destination] for source in raw_Cities for destination in raw_Cities if str(data_Cost_matrix.at[source,destination]) != 'nan'}
    raw_Minimum_Cost = {(source,destination):data_Minimum_Cost_matrix.at[source,destination] for source in raw_Cities for destination in raw_Cities if str(data_Minimum_Cost_matrix.at[source,destination]) != 'nan'}
    raw_Distance = {(source,destination):data_Distance_matrix.at[source,destination] for source in raw_Cities for destination in raw_Cities if str(data_Distance_matrix.at[source,destination]) != 'nan'}

    # 下面这种是全部卡车运输的情况
    raw_Time = {(source,destination):data_Time_matrix.at[source,destination] for source in raw_Cities for destination in raw_Cities if str(data_Time_matrix.at[source,destination]) != 'nan'}

    # 构建客户数据的dict
    raw_Handling_Cost = {depot:data_Handling_Cost.at[depot,'Handling Cost'] for depot in raw_Depots}
    raw_Demands = {customer:data_Customer_Information.at[customer,'Demand'] * i for customer in raw_Customers}
    raw_Orders = {customer:data_Customer_Information.at[customer,'Order'] for customer in raw_Customers}

    # 构建预先计算好的运输费用(Depot-Customer;Factory-Customer)
    raw_Qualify_Cost_dc = {(depot,customer):data_public_Qualify_Cost.at[depot,customer] for depot in raw_Depots for customer in raw_Customers}
    raw_Qualify_Cost_fc = {(factory,customer):data_contract_Qualify_Cost.at[factory,customer] for factory in raw_Factories for customer in raw_Customers}

    # 定义Sets and Indices
    Factories = raw_Factories
    Depots = raw_Depots
    Customers = raw_Customers
    Cities = raw_Cities

    # 定义Parameters——Multidict类型
    cost_arcs , cost = gp.multidict(raw_Cost)
    minimum_cost_arcs , minimum_cost = gp.multidict(raw_Minimum_Cost)
    distance_arcs , distance = gp.multidict(raw_Distance)
    time_arcs , time = gp.multidict(raw_Time)

    # 预先计算好的Parameters——Multidict类型
    qualify_cost_dc_arcs , qualify_cost_dc = gp.multidict(raw_Qualify_Cost_dc)
    qualify_cost_fc_arcs , qualify_cost_fc = gp.multidict(raw_Qualify_Cost_fc)

    # 定义Parameters——Dict类型
    handling_cost = raw_Handling_Cost
    demand = raw_Demands
    order = raw_Orders

    # 模型初始化及决策变量的添加
    model = gp.Model('Transportaion Network Problem')
    transportnum = model.addVars(Factories,Depots,vtype=GRB.INTEGER,name='transportnum_{f,d}')
    open_depot = model.addVars(Depots,vtype=GRB.BINARY,name='opened_depot_{d}')
    transport_type = model.addVars(Depots,Customers,vtype=GRB.BINARY,name='transport type_{d,c}')

    # 更新参数
    model.update()

    #建立目标函数
    cost_obj = model.setObjective(
    gp.quicksum(cost[f,d] * distance[f,d] * transportnum[f,d] * 0.000621371192 / 2 for f in Factories for d in Depots) + gp.quicksum(transport_type[d,c] * qualify_cost_dc[d,c] for d in Depots for c in Customers) + gp.quicksum((1-gp.quicksum(transport_type[d,c] for d in Depots)) * qualify_cost_fc[f,c] for f in Factories for c in Customers) + gp.quicksum(handling_cost[d] * transport_type[d,c] * order[c] for d in Depots for c in Customers) + 40000/52 * gp.quicksum(open_depot[d] for d in Depots),GRB.MINIMIZE
    )

    # 建立约束条件
    # 转运仓库约束：有仓库才能进行转运
    depot_flow_rule = model.addConstrs(open_depot[d] >= transport_type[d,c] for d in Depots for c in Customers)

    # 运输类型约束：一个客户只能使用一种运输方式(直送或者转运配送)
    transportation_type_rule = model.addConstrs(gp.quicksum(transport_type[d,c] for d in Depots) <= 1 for c in Customers)

    # 工厂到仓库的运输车辆约束
    f2d_vehicle_rule = model.addConstrs(transportnum[f,d] >= gp.quicksum(transport_type[d,c]*demand[c] for c in Customers) / 20000 for f in Factories for d in Depots)

    model.optimize()

    # 计算仓库的数量
    depot_num = int(var_to_dataframe(open_depot,['Depot ID','Open or Not'])[['Open or Not']].values.sum())

    return model.objVal,depot_num

In [44]:
obj_value_dict = {}
depot_num_dict = {}
for i in np.arange(0.1,3,0.04):
    obj_value,depot_num = sensitive_test(i)
    obj_value_dict[i] = obj_value
    depot_num_dict[i] = depot_num

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 351 rows, 315 columns and 1211 nonzeros
Model fingerprint: 0xd1b7339a
Variable types: 0 continuous, 315 integer (308 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  Objective range  [3e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 20344.515600
Presolve time: 0.00s
Presolved: 351 rows, 315 columns, 1211 nonzeros
Variable types: 0 continuous, 315 integer (315 binary)

Root relaxation: objective 1.578527e+04, 351 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15785.2736    0    2 20344.5156 15785.2736  22.4%     -    0s
H    0     0                    17079.624926 15785.2736  7.58%     -    0

In [29]:
def res_export(first_column_name):
    res_df = pd.DataFrame(columns=[first_column_name,'最优成本','所建仓库的数量'])
    for i in obj_value_dict.keys():
        _res_dict = [round(i,2),obj_value_dict.get(i),depot_num_dict.get(i)]
        res_df.loc[len(res_df)] = _res_dict
    return res_df

In [42]:
rate_change_df = res_export('运输费率浮动倍率').to_excel('./final data/【灵敏度】运输费率.xlsx')

In [38]:
depot_change_df = res_export('仓库费率浮动倍率').to_excel('./final data/【灵敏度】仓库费率.xlsx')

In [45]:
demand_change_df = res_export('需求浮动倍率').to_excel('./final data/【灵敏度】需求倍率.xlsx')